Getting data
rows :12477 
columns : 29
header : 'id' 'name' 'type' 'desc' 'atk' 'def' 'level' 'race' 'attribute' 'scale'
        'archetype' 'linkval' 'linkmarkers' 'image_url' 'image_url_small'
        'ban_tcg' 'ban_ocg' 'ban_goat' 'staple' 'views' 'viewsweek' 'upvotes'
        'downvotes' 'formats' 'treated_as' 'tcg_date' 'ocg_date' 'konami_id'
        'has_effect'

In [9]:
import pandas as pd
import requests
import tkinter as tk
from PIL import ImageTk, Image
from io import BytesIO
from datetime import datetime
i = 0
try:
    data = pd.read_csv("/Users/jeremyperras/Desktop/csv/database/cards.csv")
    print(data.shape[0], data.shape[1])
except Exception as error:
    print(
        f"Exception : {error.__class__.__name__} -- strerror: {error}")

12477 29


In [10]:
def GetImage():
    try :
        global i
        url = data['image_url'][i]
        image_response = requests.get(url)
        img_data = image_response.content
        return(ImageTk.PhotoImage(Image.open(BytesIO(img_data))))
    except Exception as error:
        print(
            f"Exception : {error.__class__.__name__} -- strerror: {error}")


In [11]:
def GetPrice():
    global i
    name = data['name'][i]
    respFromApi = requests.get(f"https://yugiohprices.com/api/get_card_prices/{name}")
    respFromApi.raise_for_status()
    resp = respFromApi.json()
    if respFromApi.json()["status"] == "success":
        return(respFromApi.json())
    else :
        return(0)
   

In [12]:
def clicked(labelImage:tk.Label,labelPrice:tk.Label,labelUpdate:tk.Label):
    global i
    img = GetImage()
    labelImage.configure(image=img)
    labelImage.image=img
    jsonPrice = GetPrice()
    if(jsonPrice):
        datetimeStr = jsonPrice["data"][0]["price_data"]["data"]["prices"]["updated_at"][0:19]
        datetime_object = datetime.strptime(datetimeStr, '%Y-%m-%d %H:%M:%S')
        price = jsonPrice["data"][0]["price_data"]["data"]["prices"]["average"]
        labelPrice.configure(text = f"Price: {price} $")
        labelUpdate.configure(text = f"Last up:  {datetime_object}")
    else:
        labelPrice.configure(text = "Not found") 
        labelUpdate.configure(text = "Not Found")
   


In [13]:
def Search(entry, labelImage, labelPrice, labelUpdate):
    getInput = entry.get()
    global i
    try:
        findInput = data[data["id"] == int(getInput)]
        i = findInput.index[0]
        print("test:",i)
        clicked(labelImage,labelPrice,labelUpdate)
    except Exception as error:
        print(f"Exception : {error.__class__.__name__} -- strerror: {error}")
    
    

In [14]:
def Click(*args):
    global i
    clicked(args[0], args[1],args[2])
    if(args[3]):
        if(i < data.shape[0]):
            i+=1
    else:
        if(i > 0) :
            i-=1

In [15]:
class GUI():
    def __init__(self) -> None:
        self.window = tk.Tk()
        self.inputId = tk.Entry()
        self.inputId.insert(0,"CardId: ex 02656842")
        self.labelImage = tk.Label(master=self.window, image=None)
        self.labelPrice = tk.Label(master=self.window, text=None)
        self.labelUpdate = tk.Label(master=self.window, text=None)
        self.previous = tk.Button(self.window, text="←", command=lambda:Click(self.labelImage,self.labelPrice,self.labelUpdate,0))
        self.next = tk.Button(self.window, text="→", command=lambda:Click(self.labelImage,self.labelPrice,self.labelUpdate,1))
        self.search = tk.Button(self.window, text="Search", command=lambda:Search(self.inputId,self.labelImage,self.labelPrice,self.labelUpdate))
        self.inputId.pack()
        self.labelImage.pack()
        self.labelUpdate.pack()
        self.labelPrice.pack()
        self.next.pack(side=tk.RIGHT)
        self.previous.pack(side=tk.LEFT)
        self.search.pack()
        self.window.mainloop()

Create tkinter windows

Deploy App

In [16]:
gui = GUI()
